In [ ]:
import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt

In [ ]:
glaucoma_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/rim-one-r2/Dataset-4 MIAG 2/Glaucoma and glaucoma suspicious/*.jpg")]

normal_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob("/kaggle/input/rim-one-r2/Dataset-4 MIAG 2/Normal/*.jpg")]

In [ ]:
print(len(glaucoma_images))
print(len(normal_images))

In [ ]:
new_width = 600
new_height = 600

# Create separate lists for interpolated glaucoma and normal images
interpolated_glaucoma_images = []
interpolated_normal_images = []

# Function to perform bilinear interpolation on an image
def bilinear_interpolation(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_LINEAR)

for image in glaucoma_images:
    interpolated_image = bilinear_interpolation(image, new_width, new_height)

    interpolated_glaucoma_images.append(interpolated_image)

for image in normal_images:
    # Perform bilinear interpolation
    interpolated_image = bilinear_interpolation(image, new_width, new_height)

    interpolated_normal_images.append(interpolated_image)

print("Bilinear interpolation completed.")

In [ ]:
def histogram_equalization_color(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    l_channel, a_channel, b_channel = cv2.split(lab_image)

    l_channel_equalized = cv2.equalizeHist(l_channel)

    equalized_lab_image = cv2.merge((l_channel_equalized, a_channel, b_channel))

    equalized_bgr_image = cv2.cvtColor(equalized_lab_image, cv2.COLOR_LAB2BGR)

    equalized_grayscale_image = cv2.cvtColor(equalized_bgr_image, cv2.COLOR_BGR2GRAY)

    return equalized_grayscale_image

equalized_interpolated_glaucoma_images = [histogram_equalization_color(image) for image in interpolated_glaucoma_images]

equalized_interpolated_normal_images = [histogram_equalization_color(image) for image in interpolated_normal_images]

print("Histogram Equalization and Conversion to Grayscale completed.")

In [ ]:
import random

num_images_to_display = 5

# Randomly select 'num_images_to_display' images from each list
selected_glaucoma_images = random.sample(equalized_interpolated_glaucoma_images, num_images_to_display)
selected_normal_images = random.sample(equalized_interpolated_normal_images, num_images_to_display)

fig, axes = plt.subplots(2, num_images_to_display, figsize=(15, 5))

for i in range(num_images_to_display):
    axes[0, i].imshow(cv2.cvtColor(selected_glaucoma_images[i], cv2.COLOR_BGR2RGB))
    axes[0, i].set_title("Glaucoma")
    axes[0, i].axis('off')

for i in range(num_images_to_display):
    axes[1, i].imshow(cv2.cvtColor(selected_normal_images[i], cv2.COLOR_BGR2RGB))
    axes[1, i].set_title("Normal")
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

**Augmentation**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
desired_height = 600
desired_width = 600

def augment_images(image_list, augments_per_image, max_augmented_images):
    augmented_images = []
    
    for img in image_list:
        img = cv2.resize(img, (desired_width, desired_height))
        
        img = img.reshape((1, desired_height, desired_width, 1))
        aug_iter = 0
        for aug_img in datagen.flow(img, batch_size=1):
            # Reshape the augmented image back to rank-3
            augmented_images.append(aug_img[0].reshape((desired_height, desired_width)))
            aug_iter += 1
            if aug_iter >= augments_per_image or len(augmented_images) >= max_augmented_images:
                break
                
    return augmented_images

# Initialize a list to store the augmented images
augmented_images = []
max_augmented_images = 255  # Set the maximum number of augmented images

for image in equalized_interpolated_normal_images:
    # Calculate the number of augmentations needed
    remaining_augments = max_augmented_images - len(augmented_images)
    augmentations_needed = min(2, remaining_augments)
    
    augmented = augment_images([image], augmentations_needed, remaining_augments)
    augmented_images.extend(augmented)

    if len(augmented_images) >= max_augmented_images:
        break

print("Image augmentation complete.")

In [ ]:
from sklearn.model_selection import train_test_split

# Create a list with labels (1 for glaucomous, 0 for normal)
labels = [1] * len(glaucoma_images) + [0] * len(normal_images) + [0] * len(augmented_images)

all_images = equalized_interpolated_glaucoma_images + equalized_interpolated_normal_images + augmented_images

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(all_images, labels, test_size=0.2, random_state=42, stratify=labels)

# X_train and y_train will contain the training data and labels
# X_test and y_test will contain the testing data and labels


<!-- Sequential CNN Model -->

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.callbacks import EarlyStopping

# # Create a Sequential model
# model = keras.Sequential()

# # Input layer
# model.add(layers.InputLayer(input_shape=(600, 600, 1)))

# # Convolutional layers
# for i in range(4):  
#     model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# for i in range(4):  
#     model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# for i in range(4):  
#     model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D(pool_size=(1, 1)))

# # Flatten the output
# model.add(layers.Flatten())

# # Fully connected layers with L2 regularization
# model.add(layers.Dense(512, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=l2(0.01)))
# model.add(layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', kernel_regularizer=l2(0.01)))  # Output layer with binary classification (0 or 1)

# # Early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Display model summary
# model.summary()

# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Function to define the U-Net model architecture
# def unet_model(input_size=(600, 600, 1)):
#     # Create a Sequential model
#     model = tf.keras.Sequential()

#     # Input layer
#     model.add(layers.InputLayer(input_shape=input_size))

#     # Convolutional layers
#     for i in range(4):  
#         model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
#         model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#     for i in range(4):  
#         model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
#         model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#     for i in range(4):  
#         model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
#         model.add(layers.MaxPooling2D(pool_size=(1, 1)))

#     # Flatten the output
#     model.add(layers.Flatten())

#     # Fully connected layers with L2 regularization
#     model.add(layers.Dense(512, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
#     model.add(layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)))  # Output layer with binary classification (0 or 1)

#     return model

# # Define U-Net model
# model = unet_model(input_size=(600, 600, 1))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Display model summary
# model.summary()

# # Set up early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# # Set up model checkpointing
# model_checkpoint = ModelCheckpoint('unet_glaucoma.h5', monitor='val_loss', save_best_only=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MeanIoU

# Function to define the U-Net model architecture
def unet_model(input_size=(600, 600, 1)):
    # Create a Sequential model
    model = tf.keras.Sequential()

    # Input layer
    model.add(layers.InputLayer(input_shape=input_size))

    # Convolutional layers
    for i in range(4):  
        model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    for i in range(4):  
        model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    for i in range(4):  
        model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D(pool_size=(1, 1)))

    # Flatten the output
    model.add(layers.Flatten())

    # Fully connected layers with L2 regularization
    model.add(layers.Dense(512, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)))  # Output layer with binary classification (0 or 1)

    return model

# Define U-Net model
model = unet_model(input_size=(600, 600, 1))

# Compile the model with RMSprop optimizer and Dice loss
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', MeanIoU(num_classes=2)])

# Display model summary
model.summary()

# Set up early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Set up model checkpointing
model_checkpoint = ModelCheckpoint('unet_glaucoma.h5', monitor='val_loss', save_best_only=True)





Traning the model

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

# Convert labels to NumPy arrays (ensure they are integer labels)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print(len(X_test))
print(len(X_train))
print(len(y_test))
print(len(y_train))

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),callbacks=[early_stopping])

Evaluating the model

In [ ]:
y_pred_prob = model.predict(X_test)

In [ ]:
test_loss, test_accuracy, mean_iou = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Mean IoU: {mean_iou}")


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Calculate AUC
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()